In [4]:
# !pip install googletrans==4.0.0-rc1
# !pip install pyspark py4j
# !pip install findspark
# !pip install pyspark

  Using cached pandas-2.0.2-cp311-cp311-win_amd64.whl (10.6 MB)
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
  Using cached numpy-1.24.3-cp311-cp311-win_amd64.whl (14.8 MB)


In [77]:
# from googletrans import Translator
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import pandas as pd
import pyspark
from pyspark.sql.functions import split
from pyspark.sql.functions import explode, col, trim, rtrim, ltrim
from pyspark.sql.functions import regexp_replace, regexp_extract
from pyspark.sql.functions import lower
from pyspark.sql.functions import desc
from pyspark.ml.feature import StopWordsRemover
import re

In [6]:
spark = SparkSession.builder.appName("PySpark - ContandoPalavras").getOrCreate()
sc = spark.sparkContext

spark

In [277]:
df = spark.read.text("C:/Users/cedua/CDIA - PUCSP/PROJETO PySpark - SAVINO/Contador-de-palavras---PySpark/data/fitnessandpower.txt")
df.show()
type(df)

+--------------------+
|               value|
+--------------------+
|"When you want to...|
|These powders com...|
|ARE THERE ANY DIF...|
|Yes! Whey protein...|
|This means that t...|
|DO THEY HAVE THE ...|
|No! Whey protein ...|
|DO THEY TASTE THE...|
|Nope! Whey protei...|
|DO THEY CONTAIN T...|
|No! 100 grams of ...|
|ARE THEY ABSORBED...|
|No. Whey protein ...|
|All in all, you g...|
|If you want to ha...|
|Not as well as th...|
|                    |
|For the latest ne...|
|                    |
|            Facebook|
+--------------------+
only showing top 20 rows



pyspark.sql.dataframe.DataFrame

In [87]:
########################################################################################################################

In [20]:
#### EXEMPLO 1 - MAP REDUCE
# dados=["B","B","B","C","C","C","D","D","D D"]

# input_file=sc.parallelize(dados,3)
# counts = input_file.map(lambda word:(word, 1)) /
#                      .reduceByKey(lambda a, b: a + b)

# result=counts.collect()
# print(result) 
# df=pd.DataFrame(result)
# df

[('B', 3), ('C', 3), ('D', 2), ('D D', 1)]


,0,1
0,B,3
1,C,3
2,D,2
3,D D,1


In [22]:
#### EXEMPLO 2 - MAP REDUCE
# dados=["B","B","B","C","C","C","D"]
# item=["B","D"]
# input_file=sc.parallelize(dados,3)
# counts = (input_file.map(lambda word:(word, 1) if (word in item) else (word,0)) 
#                      .reduceByKey(lambda a, b: a + b))

# result=counts.collect()
# print(result)
# df=pd.DataFrame(result)
# # df.query("qtde>0")
# df

[('B', 3), ('C', 0), ('D', 1)]


,0,1
0,B,3
1,C,0
2,D,1


In [27]:
#########################################################################################################################

In [63]:
# def translate_text(text):
#     translator = Translator()
#     translation = translator.translate(text, dest="pt")
#     return translation.text

In [71]:
# # Cria a função UDF (User Defined Function) para aplicar a tradução
# translate_udf = udf(translate_text, StringType())

# # Aplica a tradução no DataFrame usando a função UDF
# df = df.withColumn("translated_text", translate_udf(df['value']))
# # translated_df = translated_df.drop("articles_text")

# # Mostra o DataFrame com os textos traduzidos
# df.show()
# type(df)

### Tratamento - PySpark

In [278]:
#spliting the df by space " " 
df = df.select(
split(df['value'], ' ').alias('words')
)

df.show()
type(df)

+--------------------+
|               words|
+--------------------+
|["When, you, want...|
|[These, powders, ...|
|[ARE, THERE, ANY,...|
|[Yes!, Whey, prot...|
|[This, means, tha...|
|[DO, THEY, HAVE, ...|
|[No!, Whey, prote...|
|[DO, THEY, TASTE,...|
|[Nope!, Whey, pro...|
|[DO, THEY, CONTAI...|
|[No!, 100, grams,...|
|[ARE, THEY, ABSOR...|
|[No., Whey, prote...|
|[All, in, all,, y...|
|[If, you, want, t...|
|[Not, as, well, a...|
|                  []|
|[For, the, latest...|
|                  []|
|          [Facebook]|
+--------------------+
only showing top 20 rows



pyspark.sql.dataframe.DataFrame

In [279]:
stopwords_remover = StopWordsRemover(inputCol="words", outputCol="words2")

# Aplique a remoção de stopwords
df = stopwords_remover.transform(df)
df = df.drop("words")
df.show()
type(df)

+--------------------+
|              words2|
+--------------------+
|["When, want, put...|
|[powders, come, k...|
|[DIFFERENCES, WAY...|
|[Yes!, Whey, prot...|
|[means, particles...|
|[NUTRITIONAL, VAL...|
|[No!, Whey, prote...|
|      [TASTE, SAME?]|
|[Nope!, Whey, pro...|
|[CONTAIN, AMOUNT,...|
|[No!, 100, grams,...|
|   [ABSORBED, SAME?]|
|[No., Whey, prote...|
|[all,, get, make,...|
|[want, benefits, ...|
|[well, isolate,, ...|
|                  []|
|[latest, news, up...|
|                  []|
|          [Facebook]|
+--------------------+
only showing top 20 rows



pyspark.sql.dataframe.DataFrame

In [280]:
df = df.select(
    explode(col("words2")).alias("words3")
)
df.show()

+--------+
|  words3|
+--------+
|   "When|
|    want|
|     put|
|  muscle|
|increase|
| protein|
| intake,|
|     one|
|    best|
| choices|
|     buy|
|    whey|
| protein|
| powder.|
|However,|
|    even|
| picking|
|    whey|
|protein,|
|    need|
+--------+
only showing top 20 rows



In [281]:
# Removing the NULLs
df = df.where(
    col("words3") != ""
)

df.show()

+--------+
|  words3|
+--------+
|   "When|
|    want|
|     put|
|  muscle|
|increase|
| protein|
| intake,|
|     one|
|    best|
| choices|
|     buy|
|    whey|
| protein|
| powder.|
|However,|
|    even|
| picking|
|    whey|
|protein,|
|    need|
+--------+
only showing top 20 rows



In [284]:
# Removing the NULLs
df = df.where(
    col("words3") != ""
)

df.show()

+--------+
|  words4|
+--------+
|    When|
|    want|
|     put|
|  muscle|
|increase|
| protein|
|  intake|
|     one|
|    best|
| choices|
|     buy|
|    whey|
| protein|
|  powder|
| However|
|    even|
| picking|
|    whey|
| protein|
|    need|
+--------+
only showing top 20 rows



pyspark.sql.dataframe.DataFrame

In [285]:
#every word to lower case
df = df.select(
lower(
col('words4')
).alias('words5')
)

df.show()
type(df)

+--------+
|  words5|
+--------+
|    when|
|    want|
|     put|
|  muscle|
|increase|
| protein|
|  intake|
|     one|
|    best|
| choices|
|     buy|
|    whey|
| protein|
|  powder|
| however|
|    even|
| picking|
|    whey|
| protein|
|    need|
+--------+
only showing top 20 rows



pyspark.sql.dataframe.DataFrame

In [286]:
#COUNTING

df_counted = df.groupBy(
    col("words5"),
).count()

df_counted.orderBy(desc("count")).show()

+---------+-----+
|   words5|count|
+---------+-----+
|  protein|  224|
|     whey|  154|
|         |  114|
|   muscle|   41|
|      one|   32|
|       it|   32|
|      fat|   30|
|     body|   27|
|    grams|   23|
|    amino|   22|
|    study|   22|
|pinterest|   21|
|      get|   21|
|     also|   21|
| facebook|   21|
|     bcaa|   20|
|     much|   20|
|    acids|   19|
| benefits|   19|
|   powder|   17|
+---------+-----+
only showing top 20 rows



In [287]:
#RETOMANDO

df2 = df.select(
    split(
        col("words5"),
        " "
    ).alias("word6")
)
df2.show()

+----------+
|     word6|
+----------+
|    [when]|
|    [want]|
|     [put]|
|  [muscle]|
|[increase]|
| [protein]|
|  [intake]|
|     [one]|
|    [best]|
| [choices]|
|     [buy]|
|    [whey]|
| [protein]|
|  [powder]|
| [however]|
|    [even]|
| [picking]|
|    [whey]|
| [protein]|
|    [need]|
+----------+
only showing top 20 rows



In [288]:
stopwords_remover2 = StopWordsRemover(
    inputCol="word6",
    outputCol="words7"
)

df2 = stopwords_remover2.transform(df2)
df2.show()

+----------+----------+
|     word6|    words7|
+----------+----------+
|    [when]|        []|
|    [want]|    [want]|
|     [put]|     [put]|
|  [muscle]|  [muscle]|
|[increase]|[increase]|
| [protein]| [protein]|
|  [intake]|  [intake]|
|     [one]|     [one]|
|    [best]|    [best]|
| [choices]| [choices]|
|     [buy]|     [buy]|
|    [whey]|    [whey]|
| [protein]| [protein]|
|  [powder]|  [powder]|
| [however]| [however]|
|    [even]|    [even]|
| [picking]| [picking]|
|    [whey]|    [whey]|
| [protein]| [protein]|
|    [need]|    [need]|
+----------+----------+
only showing top 20 rows



In [289]:
#removing nulls
df2 = df2.select(
    (col("words7")[0]).alias("words8")
)
df2.show()

+--------+
|  words8|
+--------+
|    null|
|    want|
|     put|
|  muscle|
|increase|
| protein|
|  intake|
|     one|
|    best|
| choices|
|     buy|
|    whey|
| protein|
|  powder|
| however|
|    even|
| picking|
|    whey|
| protein|
|    need|
+--------+
only showing top 20 rows



In [290]:
df2 = df2.where(
    col("words8") != "null"
)

df2 = df2.where(
    col("words8") != ""
)

df2.show()

+--------+
|  words8|
+--------+
|    want|
|     put|
|  muscle|
|increase|
| protein|
|  intake|
|     one|
|    best|
| choices|
|     buy|
|    whey|
| protein|
|  powder|
| however|
|    even|
| picking|
|    whey|
| protein|
|    need|
|  choose|
+--------+
only showing top 20 rows



In [291]:
df_counted2 = df2.groupby(
    col("words8")
).count()

df_counted2.orderBy(desc("count")).show()

+---------+-----+
|   words8|count|
+---------+-----+
|  protein|  224|
|     whey|  154|
|   muscle|   41|
|      one|   32|
|      fat|   30|
|     body|   27|
|    grams|   23|
|    amino|   22|
|    study|   22|
|pinterest|   21|
|      get|   21|
|     also|   21|
| facebook|   21|
|     much|   20|
|     bcaa|   20|
|    acids|   19|
| benefits|   19|
|   powder|   17|
|  isolate|   17|
|     time|   17|
+---------+-----+
only showing top 20 rows

